#Use Decision Tree to Classify Iris Dataset

In [5]:
%pylab inline
from matplotlib import pyplot as plt

from sklearn import datasets, metrics, tree, cross_validation

Populating the interactive namespace from numpy and matplotlib


In [6]:
iris = datasets.load_iris()

In [7]:
# Instance of tree
myTree = tree.DecisionTreeClassifier()

In [8]:
# Fitting the model
myTree_fit = myTree.fit(iris.data, iris.target)

In [10]:
print myTree_fit

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')


In [11]:
y_pred = myTree_fit.predict(iris.data)

In [12]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [13]:
#I know we should not have done this but how did we do?
print('Number of mislabeled points %d' % (iris.target != y_pred).sum())

Number of mislabeled points 0


In [14]:
print('Absolutely ridiculous overfit score: %d' % (myTree_fit.score(iris.data, iris.target)))

Absolutely ridiculous overfit score: 1


#Confusion Matrix


In [15]:
metrics.confusion_matrix(iris.target, y_pred)

array([[50,  0,  0],
       [ 0, 50,  0],
       [ 0,  0, 50]])

In [16]:
print metrics.classification_report(iris.target, y_pred)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        50
          1       1.00      1.00      1.00        50
          2       1.00      1.00      1.00        50

avg / total       1.00      1.00      1.00       150



In [21]:
#Initialise Model
clf = tree.DecisionTreeClassifier()

#Split data into training and test sets
x_train, x_test, y_train, y_test = cross_validation.train_test_split(iris.data, iris.target, test_size=.3)

#fit model
clf.fit(x_train, y_train)

#print output for training data
print metrics.confusion_matrix(y_train, clf.predict(x_train))
print metrics.classification_report(y_train, clf.predict(x_train))

#print output for test data
print metrics.confusion_matrix(y_test, clf.predict(x_test))
print metrics.classification_report(y_test,clf.predict(x_test))

[[35  0  0]
 [ 0 37  0]
 [ 0  0 33]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00        37
          2       1.00      1.00      1.00        33

avg / total       1.00      1.00      1.00       105

[[15  0  0]
 [ 0 12  1]
 [ 0  0 17]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        15
          1       1.00      0.92      0.96        13
          2       0.94      1.00      0.97        17

avg / total       0.98      0.98      0.98        45



In [22]:
#Change parameters of 
clf.set_params(min_samples_leaf=5)
clf.set_params(max_depth=5)

#refit model
clf.fit(x_train, y_train)
print metrics.confusion_matrix(y_train, clf.predict(x_train))
print metrics.confusion_matrix(y_test, clf.predict(x_test))

[[35  0  0]
 [ 0 35  2]
 [ 0  1 32]]
[[15  0  0]
 [ 0 12  1]
 [ 0  0 17]]


In [37]:
def get_lineage(tree, feature_names):
    left     = tree.tree_.children_left
    right    = tree.tree_.children_right
    threshold= tree.tree_.threshold
    features = [feature_names[i] for i in tree.tree_.feature]
    
    idx = np.argwhere(left == -1)[:,0]
    
    def recurse(left, right, child, lineage = None):
        if lineage is None:
            lineage = [child]
        if child in left:
            parent = np.where(left == child)[0].item()
            split = 'l'
        else:
            parent = np.where(right == child)[0].item()
            split = 'r'
            
        lineage.append((parent, split, threshold[parent], features[parent]))
        
        if parent == 0:
            lineage.reverse()
            return lineage
        else:
            return recurse(left,right, parent, lineage)
        
    for child in idx:
        for node in recurse(left, right, child):
            print node

get_lineage(myTree_fit, iris.feature_names)

(0, 'l', 2.4500000476837158, 'petal length (cm)')
1
(0, 'r', 2.4500000476837158, 'petal length (cm)')
(2, 'l', 1.75, 'petal width (cm)')
(3, 'l', 4.9499998092651367, 'petal length (cm)')
(4, 'l', 1.6500000953674316, 'petal width (cm)')
5
(0, 'r', 2.4500000476837158, 'petal length (cm)')
(2, 'l', 1.75, 'petal width (cm)')
(3, 'l', 4.9499998092651367, 'petal length (cm)')
(4, 'r', 1.6500000953674316, 'petal width (cm)')
6
(0, 'r', 2.4500000476837158, 'petal length (cm)')
(2, 'l', 1.75, 'petal width (cm)')
(3, 'r', 4.9499998092651367, 'petal length (cm)')
(7, 'l', 1.5499999523162842, 'petal width (cm)')
8
(0, 'r', 2.4500000476837158, 'petal length (cm)')
(2, 'l', 1.75, 'petal width (cm)')
(3, 'r', 4.9499998092651367, 'petal length (cm)')
(7, 'r', 1.5499999523162842, 'petal width (cm)')
(9, 'l', 5.4499998092651367, 'petal length (cm)')
10
(0, 'r', 2.4500000476837158, 'petal length (cm)')
(2, 'l', 1.75, 'petal width (cm)')
(3, 'r', 4.9499998092651367, 'petal length (cm)')
(7, 'r', 1.5499999

In [25]:
import StringIO
import pydot
import pyparsing

In [38]:
dot_data = StringIO.StringIO()
tree.export_graphviz(myTree_fit, out_file = dot_data)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf('iris_decisiontree.pdf')
print '\nimage created!'



image created!
